# The Graph Data Access

In this notebook, we retrieve live Rai system data from [The Graph](https://thegraph.com/explorer/subgraph/reflexer-labs/rai-mainnet?query=Example%20query) and save it as a CSV for analysis in the [Graph Analysis](TheGraphDataAnalysis.ipynb) notebook.

In [ ]:
# import libraries
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

## Hourly stats

In [ ]:
query_header = '''
query {{
    hourlyStats(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
 timestamp
    blockNumber
    redemptionPrice {
      id
      block
      redemptionRate
      value
      
    }
    marketPriceUsd
    marketPriceEth
    globalDebt
    erc20CoinTotalSupply
'''





n = 0
safes = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats']
    print(f'results {len(s)}')
    safes.extend(s)
    print(f'total {len(safes)}')
    n += 1
    if len(s) < 1000:
        break
        
hourlyStats = pd.DataFrame(safes)

In [ ]:
hourlyStats

In [ ]:
hourlyStats.redemptionPrice.values[0]

In [ ]:
hourlyStats['redemptionPriceActual'] = hourlyStats.redemptionPrice.apply(lambda x: x['value'])

In [ ]:
hourlyStats['redemptionRateActual'] = hourlyStats.redemptionPrice.apply(lambda x: x['redemptionRate'])

In [ ]:
hourlyStats['erc20CoinTotalSupply'] = hourlyStats['erc20CoinTotalSupply'].astype(float)
hourlyStats['globalDebt'] = hourlyStats['globalDebt'].astype(float)
hourlyStats['blockNumber'] = hourlyStats['blockNumber'].astype(int)
hourlyStats['marketPriceEth'] = hourlyStats['marketPriceEth'].astype(float)
hourlyStats['marketPriceUsd'] = hourlyStats['marketPriceUsd'].astype(float)
hourlyStats['redemptionPriceActual'] = hourlyStats['redemptionPriceActual'].astype(float)
hourlyStats['redemptionRateActual'] = hourlyStats['redemptionRateActual'].astype(float)

In [ ]:
hourlyStats.describe()

In [ ]:
hourlyStats.to_csv('saved_results/RaiLiveGraphData.csv')

## Collateral Prices

In [ ]:
query_header = '''
query {{
    collateralPrices(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
    block #Block number
    timestamp # timestamp in seconds
    collateral {
      id
    } # collateral type
    safetyPrice # Price of the collateral divided by the redemptionPrice and then divided again by the safetyCRatio
    liquidationPrice # Price of the collateral divided by the redemptionPrice and then divided again by the liquidationCRatio
    value # Collateral price in denomination currency
        
'''


n = 0
safes = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['collateralPrices']
    print(f'results {len(s)}')
    safes.extend(s)
    print(f'total {len(safes)}')
    n += 1
    if len(s) < 1000:
        break
        
collateralPrices = pd.DataFrame(safes)

In [ ]:
collateralPrices['safetyPrice'] = collateralPrices['safetyPrice'].astype(float)
collateralPrices['liquidationPrice'] = collateralPrices['liquidationPrice'].astype(float)
collateralPrices['collateralPrice'] = collateralPrices['value'].astype(float)

In [ ]:
collateralPrices.describe()

In [ ]:
collateralPrices['safetyPrice'].plot(title='Price of the collateral divided by the redemptionPrice and then divided again by the safetyCRatio')         

In [ ]:
collateralPrices['liquidationPrice'].plot(title='Price of the collateral divided by the redemptionPrice and then divided again by the liquidationCRatio')

In [ ]:
collateralPrices['collateralPrice'].plot(title='Collateral price in denomination currency')

## Safes

In [ ]:
query_header = '''
query {{
    safes(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
    id # Safe ID if this Safe was created through the Safe Manager (GebSafeManager)
    collateral #Outstanding Safe debt 
    debt # Amount of collateral locked in the Safe
    owner {
        id
        safes{
            id
        }
    }
    createdAt # Block number at which this Safe was opened

'''


n = 0
safes = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['safes']
    print(f'results {len(s)}')
    safes.extend(s)
    print(f'total {len(safes)}')
    n += 1
    if len(s) < 1000:
        break
        
safes = pd.DataFrame(safes)

In [ ]:
safes['createdAt'] = safes['createdAt'].astype(float)
safes['debt'] = safes['debt'].astype(float)

In [ ]:
safes.describe()

In [ ]:
safes.owner.apply(lambda x: len([d['id'] for d in x['safes'] ])).hist()

In [ ]:
safes.debt = safes.debt.apply(float)
safes.collateral = safes.collateral.apply(float)
safes['debt_ratio'] = safes.debt/safes.collateral 

In [ ]:
safes.debt_ratio.hist()

In [ ]:
safes[safes.debt_ratio>0].debt_ratio.apply(lambda x :1/x).hist()

## Conclusion

Using The Graph, a lot of data about the Rai system can be obtained for analyzing the health of the system. With some data manipulation, these data streams could be intergrated into the Rai cadCAD model to turn it into a true decision support system.